In [1]:
import os, yaml
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

# openai.api_key  = os.environ['OPENAI_API_KEY']
openai.api_key= os.environ['OPENAI_API_KEY']

In [4]:

from langchain.requests import RequestsWrapper


from langchain.agents import create_openapi_agent
from langchain.agents.agent_toolkits import OpenAPIToolkit
from langchain.llms.openai import OpenAI
from langchain.requests import TextRequestsWrapper
from langchain.tools.json.tool import JsonSpec

with open("openapi.yaml") as f:
    data = yaml.load(f, Loader=yaml.FullLoader)
json_spec = JsonSpec(dict_=data, max_value_length=4000)


headers = {"Authorization": f"Bearer {os.getenv('OPENAI_API_KEY')}"}
openai_requests_wrapper = RequestsWrapper(headers=headers)


openapi_toolkit = OpenAPIToolkit.from_llm(
    OpenAI(temperature=0), json_spec, openai_requests_wrapper, verbose=True
)
openapi_agent_executor = create_openapi_agent(
    llm=OpenAI(temperature=0), toolkit=openapi_toolkit, verbose=True
)

In [5]:
openapi_agent_executor.run(
    "Make a post request to openai /completions. The prompt should be 'tell me a joke.'"
)



> Entering new  chain...
Action: json_explorer
Action Input: What is the base url for the API?

> Entering new  chain...
Action: json_spec_list_keys
Action Input: data
Observation: ['openapi', 'info', 'servers', 'tags', 'paths', 'components', 'x-oaiMeta']
Thought: I should look at the servers key to see what the base url is
Action: json_spec_list_keys
Action Input: data["servers"][0]
Observation: ValueError('Value at path `data["servers"][0]` is not a dict, get the value directly.')
Thought: I should get the value of the servers key
Action: json_spec_get_value
Action Input: data["servers"][0]
Observation: {'url': 'https://api.openai.com/v1'}
Thought: I now know the base url for the API
Final Answer: The base url for the API is https://api.openai.com/v1.

> Finished chain.

Observation: The base url for the API is https://api.openai.com/v1.
Thought: I should find the path for the /completions endpoint.
Action: json_explorer
Action Input: What is the path for the /completions endpoint?

"Make a POST request to https://api.openai.com/v1/completions with the parameters 'model' and 'prompt' set to the desired values."